# KW45xx and K32W1xx Load NBU image

The KW45 product family is a low-power, highly secure, single-chip wireless MCU.

It is a three-core platform, with one core for each specific domain: application, security (EdgeLock), and radio.
The radio domain features a Cortex-M3, Bluetooth LE Unit with dedicated flash. The memories integrated in
radio consist of Bluetooth LE controller stack and radio drivers. KW45 boot ROM supports remote firmware
update for the main flash (CM-33) and radio flash (CM-3).

This application note focuses on the secure update of KW45 radio firmware by loading Narrow Band Unit (NBU) image to KW45xx/K32W1xx device.

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)

Let's prepare also workspace and variables.

In [1]:
# Initialization cell
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2. Root of Trust Keys Preparation

To perform secure update, you need to establish a Root of Trust (RoT) at the beginning, which will be used later to perform update.

- The EVK board (KW45xx-EVK/K32W1xx) comes with pre-burned fuses containing the Root Key Table Hash (RKTH) in a fuse called CUST_PROD_OEMFW_AUTH_PUK, provided by NXP. The corresponding keys are stored in the EVK_keys folder.

- Generally, you can generate Root of Trust Keys (RoTKs)/Super Root Keys (SRKs) and optionally an Image Signing Key (ISK). To generate these keys, use the *nxpcrypto* application (refer to [How-to-get-keys-using-nxpcrypto](../../crypto/keys.ipynb)). By default, the script creates 4 RoTKs and 1 ISK. At least one RoTK is mandatory.

- Based on the loaded keys, a Root Key Table Hash (RKTH) value is calculated. This RKTH should be loaded into the device fuses, which is why the keys cannot be changed for the device once set.

##  3. Meaning of Container Key Blob Encryption Key

On EVK boards Container Key Blob EncryptionKey (SB3KDK) is already burnt into fuse called CUST_PROD_OEMFW_ENC_SK, you can find these values in evk_keys folder.

## 4. Prepare SB3.1 configuration file

In order to generate SB3.1 file, npximage tool is used. The nxpimage tool generates the SB3.1 file according to the configuration file.
Let's create a template for SB3.1. Modify examples according your needs. 

### 4.1 Preparing Certificate Block v2.1

In order to create signed MBI the certification block that keeps the RoT info must be prepared.
- In our case we have already prepared certification block to simplify the example (see [Certification block configuration v2.1](./inputs/cert_block_kw45_secure.yaml))
- To learn how to create a certificate blok check the whole example is presented in dedicated notebook (see [How-to-get-cert-block](../../certificate_block/cert_block_v2_1.ipynb)). 

In our case, we modified items in the SB3.1 configuration template. The ready and working template you can find in inputs folder (see [inputs-folder](./inputs/sb31_config.yaml)).

In [2]:
# Get difference of template and user YAML configuration
YamlDiffWidget("inputs/kw45xx_sb31.diffc").html

nxpimage sb31 get-template -f kw45b41z8 -o workspace/sb31_template.yaml --force 
Creating workspace/sb31_template.yaml template file.


### 4.2 SB3.1 generation

We have created certificates and keys required for the creation of SB3.1 file. Let's create a SB3.1.

In [3]:
%! nxpimage sb31 export -c inputs/sb31_config.yaml

nxpimage sb31 export -c inputs/sb31_config.yaml 
RKTH: 650d8097079ff27a3e8a2da14781b922fd8295b6c00bfa067f00e87f1a16b8b304bf710d45cbd591e2e24be83183922c
Success. (Secure binary 3.1: workspace/sb3.sb3 created.)


## 5. Device preparation

Now it's time to prepare the device. In this example we will use KW45xx-EVK/K32W1xx board.

### Board preparation

First step is to enter In System Programming (ISP) mode, this could be achieved by 3 scenarios:
- Force ISP mode by HW pin
  1) Put JP25 to (1-2)
  2) Reset the board with SW4 pressed
- Force ISP mode by debug mailbox command
  - Connect debug probe and use *nxpdebugmbox ispmode -m 1* command
- Native fall-down in case that no valid application is in chip
  - In case that chip has no valid application to boot, the ISP mode is activated natively 

Use app *nxpdevscan* to check if the device is connected to the PC in ISP mode.


**In our case, we already have burnt some fuses on EVK. Those KW45xx/K32Wxx EVK boards are distributed from NXP by already burned RoTKH and SB3KDK information as has been already mentioned earlier.**

  <img src="../../_data/img/boards/kw45xx-evk.png" alt="kw45xx-evk" height="300">

  <img src="../../_data/img/boards/k32w1xx-evk.png" alt="k32w1xx-evk" height="300">


In [4]:
# check if the device is connected and detected by PC
%! nxpdevscan

nxpdevscan 
-------- Connected NXP USB Devices --------

-------- Connected NXP UART Devices --------

Port: COM10
Type: mboot device

-------- Connected NXP SIO Devices --------

-------- Connected NXP UUU Devices --------



In [5]:
# choose com port
UART_CONNECTION = "-p com10"

## Send SB3.1 file to device

Last step is to upload SB3.1 file with NBU image to device. Command `blhost receive-sb-file` is used to update the image on CM33 flash or radio (CM3) flash via ISP.
With this command, a KW45 device receives a secure binary (SB) file, decrypts, authenticates, and programs
the image to the target memory.

In [6]:
# uploads SB3.1
%! blhost $UART_CONNECTION receive-sb-file workspace/sb3.sb3

blhost -p com10 receive-sb-file workspace/sb3.sb3 
Sending SB file
Response status = 0 (0x0) Success.
